# Milestone Project 2: Skimlit 🧾🔥

The purpose of this notebook is to build an NLP model to make reading medical abstracts easier.

The paper we're replicating (the source of the dataset that we'll be uisng) is available here: https://arxiv.org/pdf/1710.06071.pdf

And reading through the paper above, we see that the model architecture that they use to achieve their best results is avaliable here: https://arxiv.org/pdf/1612.05251.pdf

## Confirm access to a GPU

In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-a4665dcb-6d1f-c26d-5ef6-ea762d735496)


## Get the data

Since we'll be replicating the paper above (PubMed 200k RCT), let's download the dataset ther used.

We can download from the authours GitHub acount: https://github.com/Franck-Dernoncourt/pubmed-rct